In [2]:
import sys
sys.path.append('/usr/local/lib/python3.9/site-packages/')


import seaborn as sns
import bisect
import math
import matplotlib as plt
import pandas as pd
from pandas import Series,DataFrame
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import copy
import json
import lxml
from lxml import objectify 


In [76]:
#Processing DataFrames.  Split, apply, combine 
df=pd.DataFrame({
                "key1":["a","a","a","b","b"],
                "key2":["one","two","one","two","one"],
                "data1":np.arange(5),
                "data2":np.arange(5)
                })
print(df)
#groupby is series grouped by another series as the index -- confusing... 
grouped=df["data1"].groupby(df["key1"])
print(grouped.mean())
grouped=df["data1"].groupby(["one","two","a","two","one"])
print(grouped.mean())
#OR an array of Series for hierarchical indexing 
grouped=df["data1"].groupby([df["key1"],df["key2"]])
print(grouped.mean())




  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
2    a  one      2      2
3    b  two      3      3
4    b  one      4      4
key1
a    1.0
b    3.5
Name: data1, dtype: float64
a      2
one    2
two    2
Name: data1, dtype: int64
key1  key2
a     one     1
      two     1
b     one     4
      two     3
Name: data1, dtype: int64


In [77]:
print (df)

#DataFrames can be grouped using column keys
grouped=df.groupby("key1")
print(grouped.mean())
#Non-numerical data (ie key2) is skipped in mean action

#hierarchical subgroups
grouped=df.groupby(["key1","key2"])
print(grouped.mean())

for gg in grouped:
    print (gg[0])
    print (gg[1])

  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
2    a  one      2      2
3    b  two      3      3
4    b  one      4      4
      data1  data2
key1              
a       1.0    1.0
b       3.5    3.5
           data1  data2
key1 key2              
a    one       1      1
     two       1      1
b    one       4      4
     two       3      3
('a', 'one')
  key1 key2  data1  data2
0    a  one      0      0
2    a  one      2      2
('a', 'two')
  key1 key2  data1  data2
1    a  two      1      1
('b', 'one')
  key1 key2  data1  data2
4    b  one      4      4
('b', 'two')
  key1 key2  data1  data2
3    b  two      3      3


In [78]:
print (df)
#can print types or columns
print(df.dtypes)
#Double bracket preserves column index
print (df.groupby(["key1","key2"])[["data1"]].mean())


  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
2    a  one      2      2
3    b  two      3      3
4    b  one      4      4
key1     object
key2     object
data1     int64
data2     int64
dtype: object
           data1
key1 key2       
a    one       1
     two       1
b    one       4
     two       3


In [79]:
df1=pd.DataFrame(np.random.randn(5,5),index=["Kevin","Grace","Joe","Tom","Dave"],columns=["a","b","c","d","e"])
df1.iloc[2,3:]=None
print (df1)
#can map one column to another
mapping={"a":"x","b":"y","c":"x","d":"x","e":"y"}
print (df1.groupby(mapping,axis=1).sum())

#can use a python function to group based on output
#This is the name length
print (df1.groupby(len).sum())

              a         b         c         d         e
Kevin  1.694007  0.121911 -1.448885 -1.094110  1.234412
Grace -1.470225  0.300427  0.159228  2.725676 -1.434449
Joe   -1.033452 -0.781000  0.487357       NaN       NaN
Tom    2.243350  1.674060 -1.299882 -0.033178 -0.991773
Dave   0.714021  1.423635 -2.046286  0.738233  0.720318
              x         y
Kevin -0.848988  1.356323
Grace  1.414679 -1.134022
Joe   -0.546095 -0.781000
Tom    0.910290  0.682287
Dave  -0.594032  2.143953
          a         b         c         d         e
3  1.209897  0.893060 -0.812525 -0.033178 -0.991773
4  0.714021  1.423635 -2.046286  0.738233  0.720318
5  0.223782  0.422338 -1.289657  1.631566 -0.200037


In [102]:
#Aggregation means taking an array and outputting a scalar
print(df)
print()
print(df.groupby("key1")["data1"].quantile(.7))
print()

print(df.groupby("key1")["data1"].agg(["mean","std"]))
print()
#return, you can do all sorts of bonkers stuff

#different stuff per axis by dict
print(df.groupby("key1").agg({"data1":["mean","max"],"data2":["std","min"]}))


  key1 key2  data1  data2
0    a  one      0      0
1    a  two      1      1
2    a  one      2      2
3    b  two      3      3
4    b  one      4      4

key1
a    1.4
b    3.7
Name: data1, dtype: float64

      mean       std
key1                
a      1.0  1.000000
b      3.5  0.707107

     data1         data2    
      mean max       std min
key1                        
a      1.0   2  1.000000   0
b      3.5   4  0.707107   3


In [127]:
#Apply is the general aggrigate function

print()
def maxn(xdf,xnum,keyval="key1"):
    return xdf.sort_values(by=keyval)[-xnum:]
top= maxn(df,xnum=3)
#df.groupby("key1").apply(top)
#idk, return to.  also, pivot tables and cross tabulation